In [446]:
import os
import pandas as pd

# Đường dẫn đến folder chứa các file CSV
folder_path_Benign2017 = './CSVS/Benign-CSVs/Benign/Benign2017'
folder_path_Malware_Biige = './CSVS/SMSmalware-CSVs/SMSmalware/Biige'
folder_path_Malware_Fakeinst = './CSVS/SMSmalware-CSVs/SMSmalware/Fakeinst'


# Danh sách tất cả các file trong folder
file_list = os.listdir(folder_path_Benign2017)
file_list_malware_biige = os.listdir(folder_path_Malware_Biige)
file_list_malware_fakeinst = os.listdir(folder_path_Malware_Fakeinst)
# Lọc ra các file có đuôi là '.csv'
csv_files = [file for file in file_list if file.endswith('.csv')]
csv_files_malware_biige = [file for file in file_list_malware_biige if file.endswith('.csv')]
csv_files_malware_fakeinst = [file for file in file_list_malware_fakeinst if file.endswith('.csv')]

# Tạo một DataFrame để chứa dữ liệu từ tất cả các file CSV
list_df = []
all_data = pd.DataFrame()
# Đọc từng file CSV và nối dữ liệu vào DataFrame chung
i=0;
for csv_file in csv_files:
    if i < 60 :
        file_path = os.path.join(folder_path_Benign2017, csv_file)
        data = pd.read_csv(file_path, header=0)
        data.columns = data.columns.str.replace(' ', '')
        data = data.sort_values(by='Timestamp')
        data['Label'] = 1
        list_df.append(data)
        all_data = pd.concat([all_data, data], ignore_index=True) 
        i = i + 1

for csv_file in csv_files_malware_biige : 
    file_path = os.path.join(folder_path_Malware_Biige, csv_file)
    data = pd.read_csv(file_path, header=0)
    data.columns = data.columns.str.replace(' ', '')
    data = data.sort_values(by='Timestamp')
    data['Label'] = 0
    list_df.append(data)
    all_data = pd.concat([all_data, data], ignore_index=True)
for csv_file in csv_files_malware_fakeinst : 
    file_path = os.path.join(folder_path_Malware_Fakeinst, csv_file)
    data = pd.read_csv(file_path, header=0)
    data.columns = data.columns.str.replace(' ', '')
    data = data.sort_values(by='Timestamp')
    data['Label'] = 0
    list_df.append(data)
    all_data = pd.concat([all_data, data], ignore_index=True)



# Hiển thị DataFrame chứa dữ liệu từ tất cả các file CSV
print(all_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84768 entries, 0 to 84767
Data columns (total 85 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   FlowID                   84767 non-null  object 
 1   SourceIP                 84768 non-null  object 
 2   SourcePort               84768 non-null  int64  
 3   DestinationIP            84768 non-null  object 
 4   DestinationPort          84768 non-null  int64  
 5   Protocol                 84768 non-null  int64  
 6   Timestamp                84768 non-null  object 
 7   FlowDuration             84768 non-null  int64  
 8   TotalFwdPackets          84768 non-null  int64  
 9   TotalBackwardPackets     84768 non-null  int64  
 10  TotalLengthofFwdPackets  84768 non-null  float64
 11  TotalLengthofBwdPackets  84768 non-null  float64
 12  FwdPacketLengthMax       84768 non-null  float64
 13  FwdPacketLengthMin       84768 non-null  float64
 14  FwdPacketLengthMean   

In [447]:
all_data['Label'].value_counts()

Label
0    49124
1    35644
Name: count, dtype: int64

In [448]:
def is_valid_format(timestamp, format='%d/%m/%Y %H:%M:%S'):
    try:
        pd.to_datetime(timestamp, format='%d/%m/%Y %H:%M:%S')
        return True
    except ValueError:
        return False
    
list_df_5_minutes = []
#Tạo list graph
for df in list_df: 
    i = 0
    timeStamp = df['Timestamp'].iloc[i] 
    # Chuyển đổi timestamp thành đối tượng datetime và cộng 5 phút
    
    while not is_valid_format(timeStamp, format='%d/%m/%Y %H:%M:%S') :
        i = i + 1
        timeStamp = df['Timestamp'].iloc[i]

    datetime = pd.to_datetime(timeStamp, format='%d/%m/%Y %H:%M:%S') + pd.Timedelta(minutes=5)
    new_df = pd.DataFrame(columns=all_data.columns)
    for row in df.itertuples(index=False):
        try:
            date_time_of_row = pd.to_datetime(row.Timestamp, format='%d/%m/%Y %H:%M:%S')
        except ValueError:
            # Bỏ qua row nếu không thể chuyển đổi thành đối tượng datetime
            continue
        if(date_time_of_row <= datetime) : 
            new_df.loc[len(new_df)] = list(row)
        elif not new_df.empty: 
            list_df_5_minutes.append(new_df)
            new_df = pd.DataFrame(columns=all_data.columns)
            datetime = datetime + pd.Timedelta(minutes=5)

len(list_df_5_minutes)

271

In [449]:
def convert_ip_label(df):
    le_columns = ['SourceIP', 'DestinationIP']
    ip_list = list(df['SourceIP'].unique()) + list(df['DestinationIP'].unique())
    ip_set = list(set(ip_list))

    for column in le_columns:
        list_unique = list(df[column].unique())
        for val in list_unique:
            df.loc[df[column] == val, column] = ip_set.index(val)

    df['DestinationIP'] = df['DestinationIP'].astype(int)
    df['SourceIP'] = df['SourceIP'].astype(int)
    return df

In [450]:
import networkx as nx
import matplotlib.pyplot as plt
list_graph = []
list_label = []
for df in list_df_5_minutes:
    df = convert_ip_label(df)
    graph = nx.from_pandas_edgelist(df, 'SourceIP', 'DestinationIP',
                                     create_using=nx.MultiGraph(), edge_attr=df.drop(columns=['FlowID', 'SourceIP', 'DestinationIP', 'SourcePort', 'DestinationPort', 'Label', 
                                                                                              'Timestamp', 'PacketLengthStd', 'CWEFlagCount', 'Down/UpRatio']).columns.values.tolist())
    list_graph.append(graph)
    count_value = len(df['Label'].unique())
    if count_value == 1: 
        label = df['Label'].unique()[0]
    else: 
        label = 0
    list_label.append(label)
    


In [463]:
import torch
import torch.nn as nn
from torch_geometric.data import Data, DataLoader
from torch_geometric.loader import  DataLoader

from torch_geometric.nn import GCNConv, global_mean_pool, global_max_pool, global_add_pool
from sklearn.model_selection import train_test_split
import torch.nn.functional as F

# Định nghĩa mô hình GCN
class GCN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, output_dim)
        self.fc = nn.Linear(output_dim, 2)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = self.conv1(x, edge_index)
        x = torch.relu(x)
        x = F.dropout(x, p=0.2, training=self.training)
        x = self.conv2(x, edge_index)
        x = global_add_pool(x, batch)
        x = self.fc(x)
        return x


In [464]:
# Chuyển đổi và chia tập dữ liệu 
def graph_to_pyg_data(graph, label):
    edge_index = torch.tensor(list(graph.edges)).t().contiguous()[:-1]
    x = torch.tensor([list(graph.edges[edge].values()) for edge in graph.edges], dtype=torch.float)
    y = torch.tensor([label], dtype=torch.long)
    return Data(x=x, edge_index=edge_index, y=y)
list_graph_labeled = list(zip(list_graph, list_label))
list_data = []
for graph, label in list_graph_labeled:
    list_data.append(graph_to_pyg_data(graph, label))
    
# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
train_data, test_data = train_test_split(list_data, test_size=0.2, random_state=42)
# Tạo DataLoader cho tập huấn luyện và tập kiểm tra
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False)

In [466]:
import torch
import torch.nn as nn
import torch.optim as optim
# Huấn luyện mô hình
def train(model, train_loader, criterion, optimizer, device):
    model.train()
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, data.y)
        loss.backward()
        optimizer.step()

# Đánh giá mô hình
def evaluate(model, loader, criterion, device):
    model.eval()
    total_loss = 0.0
    correct = 0
    total_samples = 0

    with torch.no_grad():
        for data in loader:
            data = data.to(device)
            output = model(data)
            loss = criterion(output, data.y)
            total_loss += loss.item()
            _, predicted = torch.max(output, 1)
            correct += (predicted == data.y).sum().item()
            total_samples += data.y.size(0)

    accuracy = correct / total_samples
    average_loss = total_loss / len(loader)

    return accuracy, average_loss

# Thiết lập các tham số
input_dim = 75 # Thay thế bằng kích thước của vectơ đặc trưng đỉnh
hidden_dim = 64
output_dim = 2
lr = 0.001
epochs = 100

# Tạo mô hình và các thành phần khác
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCN(input_dim, hidden_dim, output_dim).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

# Huấn luyện và kiểm thử mô hình
for epoch in range(epochs):
    train(model, train_loader, criterion, optimizer, device)
    train_accuracy, train_loss = evaluate(model, train_loader, criterion, device)
    test_accuracy, test_loss = evaluate(model, test_loader, criterion, device)
    print(f'Epoch {epoch + 1}/{epochs}, Train Loss: {train_loss:.4f}, Test Acc: {test_accuracy * 100:.2f}%')


Epoch 1/100, Train Loss: 131236051.5000, Test Acc: 52.73%
Epoch 2/100, Train Loss: 93359564.3750, Test Acc: 54.55%
Epoch 3/100, Train Loss: 43698913.7500, Test Acc: 61.82%
Epoch 4/100, Train Loss: 42138244.5000, Test Acc: 54.55%
Epoch 5/100, Train Loss: 26227143.7500, Test Acc: 67.27%
Epoch 6/100, Train Loss: 44898263.2500, Test Acc: 65.45%
Epoch 7/100, Train Loss: 30341275.0000, Test Acc: 58.18%
Epoch 8/100, Train Loss: 19422307.5000, Test Acc: 60.00%
Epoch 9/100, Train Loss: 19529330.2500, Test Acc: 61.82%
Epoch 10/100, Train Loss: 23706362.2500, Test Acc: 58.18%
Epoch 11/100, Train Loss: 17668154.0000, Test Acc: 61.82%
Epoch 12/100, Train Loss: 15745110.5000, Test Acc: 63.64%
Epoch 13/100, Train Loss: 14909510.4375, Test Acc: 61.82%
Epoch 14/100, Train Loss: 18660435.0000, Test Acc: 61.82%
Epoch 15/100, Train Loss: 12060941.5000, Test Acc: 60.00%
Epoch 16/100, Train Loss: 9985016.7812, Test Acc: 60.00%
Epoch 17/100, Train Loss: 12438553.8750, Test Acc: 54.55%
Epoch 18/100, Train Los